# Imports

In [5]:
import pandas as pd
import numpy as np
import random
import pickle
import math
import re
import collections
import itertools
import sklearn
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
import keras.utils as ku 
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dropout, Bidirectional, Embedding, LSTM, Dense

# http://flask.pocoo.org/docs/1.0/cli/

Using TensorFlow backend.


# Data

In [2]:
df = pd.read_csv('data/all_cocktail_data.csv', index_col=0)
df.fillna('', inplace=True)
df = df[[
'Question1',
'Question2',
'Question3',
'Question4',
'Liquor',
'Ingredient1',
'Ingredient2',
'Ingredient3',
'Ingredient4',
'Ingredient5',
'Ingredient6',
'Ingredient7',
'Ingredient8']]
df.reset_index(inplace=True, drop=True)
df = df.apply(lambda x: x.astype(str).str.lower())
df.head()

,Question1,Question2,Question3,Question4,Liquor,Ingredient1,Ingredient2,Ingredient3,Ingredient4,Ingredient5,Ingredient6,Ingredient7,Ingredient8
0,a,a,d,e,bourbon,gingerale,,,,,,,
1,a,c,d,a,scotch,soda,,,,,,,
2,a,a,d,c,seagram 7,7up,,,,,,,
3,a,b,d,e,whiskey,water,,,,,,,
4,d,c,d,a,light rum,coke,,,,,,,


### Final Liquor Touch-Ups

In [3]:
# Replace specific whiskey and bourbon names with generic alcohol types
df.loc[df.query('Question1=="a" & Liquor in ["early times","southern comfort"]').index, 'Liquor'] = 'bourbon'
df.loc[df.query('Question1=="a" & Liquor in ["jack daniels"]').index, 'Liquor'] = 'whiskey'

In [4]:
# Create word phrases joined by '-'
for c in df.columns:
    df[c] = df[c].apply(lambda x: '-'.join(x.strip().split()))

In [5]:
df.head()

,Question1,Question2,Question3,Question4,Liquor,Ingredient1,Ingredient2,Ingredient3,Ingredient4,Ingredient5,Ingredient6,Ingredient7,Ingredient8
0,a,a,d,e,bourbon,gingerale,,,,,,,
1,a,c,d,a,scotch,soda,,,,,,,
2,a,a,d,c,seagram-7,7up,,,,,,,
3,a,b,d,e,whiskey,water,,,,,,,
4,d,c,d,a,light-rum,coke,,,,,,,


### List of Documents for Keras Pre-Processing

In [6]:
# Create list of 'documents' for Keras pre-processing
full_txt = df.to_string(header=False,
                  index=False,
                  index_names=False).split('\n')
docs = [','.join(x.split()) for x in full_txt]

In [7]:
docs

['a,a,d,e,bourbon,gingerale',
 'a,c,d,a,scotch,soda',
 'a,a,d,c,seagram-7,7up',
 'a,b,d,e,whiskey,water',
 'd,c,d,a,light-rum,coke',
 'd,d,d,a,light-rum,coke',
 'b,d,d,d,gin,tonic',
 'a,c,d,a,bourbon,soda-gingerale',
 'b,d,d,a,gin,soda',
 'a,d,d,a,bourbon,soda',
 'b,d,d,a,vodka,tomato-juice',
 'c,d,d,a,tequila,tomato-juice',
 'b,d,d,a,vodka,grapefruit-juice',
 'b,d,d,a,vodka,grapefruit-juice',
 'b,a,d,a,vodka,orange-juice',
 'b,a,c,a,vodka,galliano,orange-juice',
 'c,a,c,a,tequila,orange-juice,grenadine',
 'b,d,d,a,vodka,cranberry-juice',
 'b,d,d,b,vodka,cranberry-grapefruit',
 'b,d,d,b,vodka,cranberry-pineapple',
 'b,a,d,b,vodka,cranberry-orange',
 'b,a,c,b,vodka,peach-schnapps,cranberry-orange',
 'b,a,d,a,sloe-gin,orange-juice',
 'b,a,c,a,sloe-gin,southern-comfort,orange-juice',
 'd,a,c,a,amaretto,orange-juice,soda',
 'b,a,c,a,vodka,midori,orange-juice',
 'd,a,d,a,peach-schnapps,orange-juice',
 'd,a,c,a,peach-schnapps,vodka,orange-juice',
 'a,b,c,c,brandy,brown-creme-de-cacao,cream',

### Permutations

In [8]:
# create new list of docs with all permutations of the drinks to bolster the vectorization and recipe list
new_docs = []
for d in docs:
    temp_ans = d.split(',')[0:5] # ex: ['a', 'a', 'd', 'e', 'vodka']
    perm_list = list(itertools.permutations(d.split(',')[5:]))
    # ex: [('cream-of-coconut-syrup', 'pineapple-juice'), ('pineapple-juice', 'cream-of-coconut-syrup')]
    # iterate through perm_list to create new doc elements, up to three
    i = 0
    for p in range(len(perm_list)):
        i += 1
        temp_new = temp_ans + (list(perm_list[p]))
        temp_new = ','.join(temp_new)
        # ex: 'b,d,c,a,vodka,cream-of-coconut-syrup,pineapple-juice'
        new_docs.append(temp_new)
        if i > 3:
            break
#         print(temp_new)

In [9]:
new_docs

['a,a,d,e,bourbon,gingerale',
 'a,c,d,a,scotch,soda',
 'a,a,d,c,seagram-7,7up',
 'a,b,d,e,whiskey,water',
 'd,c,d,a,light-rum,coke',
 'd,d,d,a,light-rum,coke',
 'b,d,d,d,gin,tonic',
 'a,c,d,a,bourbon,soda-gingerale',
 'b,d,d,a,gin,soda',
 'a,d,d,a,bourbon,soda',
 'b,d,d,a,vodka,tomato-juice',
 'c,d,d,a,tequila,tomato-juice',
 'b,d,d,a,vodka,grapefruit-juice',
 'b,d,d,a,vodka,grapefruit-juice',
 'b,a,d,a,vodka,orange-juice',
 'b,a,c,a,vodka,galliano,orange-juice',
 'b,a,c,a,vodka,orange-juice,galliano',
 'c,a,c,a,tequila,orange-juice,grenadine',
 'c,a,c,a,tequila,grenadine,orange-juice',
 'b,d,d,a,vodka,cranberry-juice',
 'b,d,d,b,vodka,cranberry-grapefruit',
 'b,d,d,b,vodka,cranberry-pineapple',
 'b,a,d,b,vodka,cranberry-orange',
 'b,a,c,b,vodka,peach-schnapps,cranberry-orange',
 'b,a,c,b,vodka,cranberry-orange,peach-schnapps',
 'b,a,d,a,sloe-gin,orange-juice',
 'b,a,c,a,sloe-gin,southern-comfort,orange-juice',
 'b,a,c,a,sloe-gin,orange-juice,southern-comfort',
 'd,a,c,a,amaretto,orang

## Create Tokenizer

In [10]:
# Tokenize words to create word vectors
tokenizer = Tokenizer(num_words=None, 
                      filters='!"#$%*+,./:;<=>?@[\\]^_`{|}~\t\n', 
                      lower=True, split=' ', 
                      char_level=False, 
                      oov_token=None, 
                      document_count=0)

tokenizer.fit_on_texts(new_docs)
print('Number of documents: ', tokenizer.document_count)
total_words = len(tokenizer.word_index)+1
print('Number of words in dictionary: ', total_words)

Number of documents:  5065
Number of words in dictionary:  1272


In [11]:
# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [13]:
tokenizer.texts_to_sequences(new_docs)

[[1, 1, 3, 5, 7, 1233],
 [1, 4, 3, 1, 148, 149],
 [1, 1, 3, 4, 1234, 137],
 [1, 2, 3, 5, 14, 83],
 [3, 4, 3, 1, 70, 140],
 [3, 3, 3, 1, 70, 140],
 [2, 3, 3, 3, 8, 429],
 [1, 4, 3, 1, 7, 1235],
 [2, 3, 3, 1, 8, 149],
 [1, 3, 3, 1, 7, 149],
 [2, 3, 3, 1, 12, 281],
 [4, 3, 3, 1, 20, 281],
 [2, 3, 3, 1, 12, 38],
 [2, 3, 3, 1, 12, 38],
 [2, 1, 3, 1, 12, 10],
 [2, 1, 4, 1, 12, 71, 10],
 [2, 1, 4, 1, 12, 10, 71],
 [4, 1, 4, 1, 20, 10, 19],
 [4, 1, 4, 1, 20, 19, 10],
 [2, 3, 3, 1, 12, 27],
 [2, 3, 3, 2, 12, 1236],
 [2, 3, 3, 2, 12, 1237],
 [2, 1, 3, 2, 12, 1106],
 [2, 1, 4, 2, 12, 42, 1106],
 [2, 1, 4, 2, 12, 1106, 42],
 [2, 1, 3, 1, 69, 10],
 [2, 1, 4, 1, 69, 76, 10],
 [2, 1, 4, 1, 69, 10, 76],
 [3, 1, 4, 1, 25, 10, 149],
 [3, 1, 4, 1, 25, 149, 10],
 [2, 1, 4, 1, 12, 146, 10],
 [2, 1, 4, 1, 12, 10, 146],
 [3, 1, 3, 1, 42, 10],
 [3, 1, 4, 1, 42, 12, 10],
 [3, 1, 4, 1, 42, 10, 12],
 [1, 2, 4, 4, 22, 1111, 89],
 [1, 2, 4, 4, 22, 89, 1111],
 [4, 2, 4, 4, 71, 64, 89],
 [4, 2, 4, 4, 71, 89, 64],
 [

### Tokenize Documents and Create Input Sequences

In [14]:
input_sequences = []
for line in new_docs:
    token_list = tokenizer.texts_to_sequences([line])[0]
    print(line)
    print(token_list)
    for i in range(4, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        print(n_gram_sequence)
        input_sequences.append(n_gram_sequence)

a,a,d,e,bourbon,gingerale
[1, 1, 3, 5, 7, 1233]
[1, 1, 3, 5, 7]
[1, 1, 3, 5, 7, 1233]
a,c,d,a,scotch,soda
[1, 4, 3, 1, 148, 149]
[1, 4, 3, 1, 148]
[1, 4, 3, 1, 148, 149]
a,a,d,c,seagram-7,7up
[1, 1, 3, 4, 1234, 137]
[1, 1, 3, 4, 1234]
[1, 1, 3, 4, 1234, 137]
a,b,d,e,whiskey,water
[1, 2, 3, 5, 14, 83]
[1, 2, 3, 5, 14]
[1, 2, 3, 5, 14, 83]
d,c,d,a,light-rum,coke
[3, 4, 3, 1, 70, 140]
[3, 4, 3, 1, 70]
[3, 4, 3, 1, 70, 140]
d,d,d,a,light-rum,coke
[3, 3, 3, 1, 70, 140]
[3, 3, 3, 1, 70]
[3, 3, 3, 1, 70, 140]
b,d,d,d,gin,tonic
[2, 3, 3, 3, 8, 429]
[2, 3, 3, 3, 8]
[2, 3, 3, 3, 8, 429]
a,c,d,a,bourbon,soda-gingerale
[1, 4, 3, 1, 7, 1235]
[1, 4, 3, 1, 7]
[1, 4, 3, 1, 7, 1235]
b,d,d,a,gin,soda
[2, 3, 3, 1, 8, 149]
[2, 3, 3, 1, 8]
[2, 3, 3, 1, 8, 149]
a,d,d,a,bourbon,soda
[1, 3, 3, 1, 7, 149]
[1, 3, 3, 1, 7]
[1, 3, 3, 1, 7, 149]
b,d,d,a,vodka,tomato-juice
[2, 3, 3, 1, 12, 281]
[2, 3, 3, 1, 12]
[2, 3, 3, 1, 12, 281]
c,d,d,a,tequila,tomato-juice
[4, 3, 3, 1, 20, 281]
[4, 3, 3, 1, 20]
[4, 3, 3, 1, 20

[2, 3, 4, 1, 12, 13, 15]
[2, 3, 4, 1, 12]
[2, 3, 4, 1, 12, 13]
[2, 3, 4, 1, 12, 13, 15]
b,c,c,d,vodka,chambord,sour-mix
[2, 4, 4, 3, 12, 1115, 60]
[2, 4, 4, 3, 12]
[2, 4, 4, 3, 12, 1115]
[2, 4, 4, 3, 12, 1115, 60]
b,c,c,d,vodka,sour-mix,chambord
[2, 4, 4, 3, 12, 60, 1115]
[2, 4, 4, 3, 12]
[2, 4, 4, 3, 12, 60]
[2, 4, 4, 3, 12, 60, 1115]
d,b,c,f,kahlua,baileys,amaretto
[3, 2, 4, 6, 125, 183, 25]
[3, 2, 4, 6, 125]
[3, 2, 4, 6, 125, 183]
[3, 2, 4, 6, 125, 183, 25]
d,b,c,f,kahlua,amaretto,baileys
[3, 2, 4, 6, 125, 25, 183]
[3, 2, 4, 6, 125]
[3, 2, 4, 6, 125, 25]
[3, 2, 4, 6, 125, 25, 183]
d,b,b,f,kahlua,baileys,amaretto,vodka
[3, 2, 2, 6, 125, 183, 25, 12]
[3, 2, 2, 6, 125]
[3, 2, 2, 6, 125, 183]
[3, 2, 2, 6, 125, 183, 25]
[3, 2, 2, 6, 125, 183, 25, 12]
d,b,b,f,kahlua,baileys,vodka,amaretto
[3, 2, 2, 6, 125, 183, 12, 25]
[3, 2, 2, 6, 125]
[3, 2, 2, 6, 125, 183]
[3, 2, 2, 6, 125, 183, 12]
[3, 2, 2, 6, 125, 183, 12, 25]
d,b,b,f,kahlua,amaretto,baileys,vodka
[3, 2, 2, 6, 125, 25, 183, 12]
[3, 

a,d,b,e,korbel-spiced-brandy,gold-rum,pineapple-juice,roses-sweetened-lime-juice
[1, 3, 2, 5, 54, 150, 18, 461]
[1, 3, 2, 5, 54]
[1, 3, 2, 5, 54, 150]
[1, 3, 2, 5, 54, 150, 18]
[1, 3, 2, 5, 54, 150, 18, 461]
a,d,c,d,whiskey,frozen-limeade-concentrate,beer
[1, 3, 4, 3, 14, 1120, 1121]
[1, 3, 4, 3, 14]
[1, 3, 4, 3, 14, 1120]
[1, 3, 4, 3, 14, 1120, 1121]
a,d,c,d,whiskey,beer,frozen-limeade-concentrate
[1, 3, 4, 3, 14, 1121, 1120]
[1, 3, 4, 3, 14]
[1, 3, 4, 3, 14, 1121]
[1, 3, 4, 3, 14, 1121, 1120]
a,b,b,c,bourbon,small-cinnamon-stick,slice-lemon-peel,hot-cider
[1, 2, 2, 4, 7, 462, 463, 464]
[1, 2, 2, 4, 7]
[1, 2, 2, 4, 7, 462]
[1, 2, 2, 4, 7, 462, 463]
[1, 2, 2, 4, 7, 462, 463, 464]
a,b,b,c,bourbon,small-cinnamon-stick,hot-cider,slice-lemon-peel
[1, 2, 2, 4, 7, 462, 464, 463]
[1, 2, 2, 4, 7]
[1, 2, 2, 4, 7, 462]
[1, 2, 2, 4, 7, 462, 464]
[1, 2, 2, 4, 7, 462, 464, 463]
a,b,b,c,bourbon,slice-lemon-peel,small-cinnamon-stick,hot-cider
[1, 2, 2, 4, 7, 463, 462, 464]
[1, 2, 2, 4, 7]
[1, 2, 2, 4

[1, 2, 1, 4, 22, 471, 57]
[1, 2, 1, 4, 22, 471, 57, 59]
[1, 2, 1, 4, 22, 471, 57, 59, 156]
a,b,a,c,brandy,brunch,half-&-half,chocolate-syrup,dark-creme-de-cacao
[1, 2, 1, 4, 22, 471, 57, 156, 59]
[1, 2, 1, 4, 22]
[1, 2, 1, 4, 22, 471]
[1, 2, 1, 4, 22, 471, 57]
[1, 2, 1, 4, 22, 471, 57, 156]
[1, 2, 1, 4, 22, 471, 57, 156, 59]
a,d,a,g,brandy,vanilla-extract,half-&-half,vanilla-ice-cream,crushed-ice
[1, 3, 1, 29, 22, 195, 57, 35, 21]
[1, 3, 1, 29, 22]
[1, 3, 1, 29, 22, 195]
[1, 3, 1, 29, 22, 195, 57]
[1, 3, 1, 29, 22, 195, 57, 35]
[1, 3, 1, 29, 22, 195, 57, 35, 21]
a,d,a,g,brandy,vanilla-extract,half-&-half,crushed-ice,vanilla-ice-cream
[1, 3, 1, 29, 22, 195, 57, 21, 35]
[1, 3, 1, 29, 22]
[1, 3, 1, 29, 22, 195]
[1, 3, 1, 29, 22, 195, 57]
[1, 3, 1, 29, 22, 195, 57, 21]
[1, 3, 1, 29, 22, 195, 57, 21, 35]
a,d,a,g,brandy,vanilla-extract,vanilla-ice-cream,half-&-half,crushed-ice
[1, 3, 1, 29, 22, 195, 35, 57, 21]
[1, 3, 1, 29, 22]
[1, 3, 1, 29, 22, 195]
[1, 3, 1, 29, 22, 195, 35]
[1, 3, 1, 29,

[1, 2, 1, 4, 7, 141, 9, 18, 15]
c,d,b,a,tequila,margarita-mix,triple-sec,crushed-ice
[4, 3, 2, 1, 20, 67, 15, 21]
[4, 3, 2, 1, 20]
[4, 3, 2, 1, 20, 67]
[4, 3, 2, 1, 20, 67, 15]
[4, 3, 2, 1, 20, 67, 15, 21]
c,d,b,a,tequila,margarita-mix,crushed-ice,triple-sec
[4, 3, 2, 1, 20, 67, 21, 15]
[4, 3, 2, 1, 20]
[4, 3, 2, 1, 20, 67]
[4, 3, 2, 1, 20, 67, 21]
[4, 3, 2, 1, 20, 67, 21, 15]
c,d,b,a,tequila,triple-sec,margarita-mix,crushed-ice
[4, 3, 2, 1, 20, 15, 67, 21]
[4, 3, 2, 1, 20]
[4, 3, 2, 1, 20, 15]
[4, 3, 2, 1, 20, 15, 67]
[4, 3, 2, 1, 20, 15, 67, 21]
c,d,b,a,tequila,triple-sec,crushed-ice,margarita-mix
[4, 3, 2, 1, 20, 15, 21, 67]
[4, 3, 2, 1, 20]
[4, 3, 2, 1, 20, 15]
[4, 3, 2, 1, 20, 15, 21]
[4, 3, 2, 1, 20, 15, 21, 67]
a,d,a,g,canadian-mist,triple-sec,bitters,sugar,orange-wedge
[1, 3, 1, 29, 31, 15, 41, 46, 117]
[1, 3, 1, 29, 31]
[1, 3, 1, 29, 31, 15]
[1, 3, 1, 29, 31, 15, 41]
[1, 3, 1, 29, 31, 15, 41, 46]
[1, 3, 1, 29, 31, 15, 41, 46, 117]
a,d,a,g,canadian-mist,triple-sec,bitters,orang

...

[1, 3, 1, 2, 16, 107, 884]
[1, 3, 1, 2, 16, 107, 884, 886]
[1, 3, 1, 2, 16, 107, 884, 886, 885]
[1, 3, 1, 2, 16, 107, 884, 886, 885, 343]
a,d,a,b,whisky,junipero-gin,byrrh-grand-quinquina,bittercube-corazon-bitters,the-bitter-truth-orange-bitters,anchor-christmas-ale-2016*
[1, 3, 1, 2, 16, 107, 884, 886, 343, 885]
[1, 3, 1, 2, 16]
[1, 3, 1, 2, 16, 107]
[1, 3, 1, 2, 16, 107, 884]
[1, 3, 1, 2, 16, 107, 884, 886]
[1, 3, 1, 2, 16, 107, 884, 886, 343]
[1, 3, 1, 2, 16, 107, 884, 886, 343, 885]
a,b,b,d,whisky,luxardo-amaro-abano,punt-e-mes,angostura-bitters
[1, 2, 2, 3, 16, 65, 207, 24]
[1, 2, 2, 3, 16]
[1, 2, 2, 3, 16, 65]
[1, 2, 2, 3, 16, 65, 207]
[1, 2, 2, 3, 16, 65, 207, 24]
a,b,b,d,whisky,luxardo-amaro-abano,angostura-bitters,punt-e-mes
[1, 2, 2, 3, 16, 65, 24, 207]
[1, 2, 2, 3, 16]
[1, 2, 2, 3, 16, 65]
[1, 2, 2, 3, 16, 65, 24]
[1, 2, 2, 3, 16, 65, 24, 207]
a,b,b,d,whisky,punt-e-mes,luxardo-amaro-abano,angostura-bitters
[1, 2, 2, 3, 16, 207, 65, 24]
[1, 2, 2, 3, 16]
[1, 2, 2, 3, 16, 207]

[3, 3, 2, 2, 11, 123, 935, 934]
d,d,b,b,rum,ancho-reyes,barsol-perfecto-amor-(aperitif-wine),house-made-roasted-banana-bitters
[3, 3, 2, 2, 11, 934, 123, 935]
[3, 3, 2, 2, 11]
[3, 3, 2, 2, 11, 934]
[3, 3, 2, 2, 11, 934, 123]
[3, 3, 2, 2, 11, 934, 123, 935]
d,d,b,b,rum,ancho-reyes,house-made-roasted-banana-bitters,barsol-perfecto-amor-(aperitif-wine)
[3, 3, 2, 2, 11, 934, 935, 123]
[3, 3, 2, 2, 11]
[3, 3, 2, 2, 11, 934]
[3, 3, 2, 2, 11, 934, 935]
[3, 3, 2, 2, 11, 934, 935, 123]
a,b,b,e,cognac,pear-brandy,dry-vermouth,dolin-genepy
[1, 2, 2, 5, 26, 377, 40, 936]
[1, 2, 2, 5, 26]
[1, 2, 2, 5, 26, 377]
[1, 2, 2, 5, 26, 377, 40]
[1, 2, 2, 5, 26, 377, 40, 936]
a,b,b,e,cognac,pear-brandy,dolin-genepy,dry-vermouth
[1, 2, 2, 5, 26, 377, 936, 40]
[1, 2, 2, 5, 26]
[1, 2, 2, 5, 26, 377]
[1, 2, 2, 5, 26, 377, 936]
[1, 2, 2, 5, 26, 377, 936, 40]
a,b,b,e,cognac,dry-vermouth,pear-brandy,dolin-genepy
[1, 2, 2, 5, 26, 40, 377, 936]
[1, 2, 2, 5, 26]
[1, 2, 2, 5, 26, 40]
[1, 2, 2, 5, 26, 40, 377]
[1, 2, 2,

[1, 1, 1, 6, 16]
[1, 1, 1, 6, 16, 55]
[1, 1, 1, 6, 16, 55, 28]
[1, 1, 1, 6, 16, 55, 28, 1001]
[1, 1, 1, 6, 16, 55, 28, 1001, 17]
[1, 1, 1, 6, 16, 55, 28, 1001, 17, 1002]
a,a,a,f,whisky,luxardo-amaretto-di-saschira,fresh-lemon-juice,sparkling-apple-juice,seltzer,simple-syrup
[1, 1, 1, 6, 16, 55, 28, 1001, 1002, 17]
[1, 1, 1, 6, 16]
[1, 1, 1, 6, 16, 55]
[1, 1, 1, 6, 16, 55, 28]
[1, 1, 1, 6, 16, 55, 28, 1001]
[1, 1, 1, 6, 16, 55, 28, 1001, 1002]
[1, 1, 1, 6, 16, 55, 28, 1001, 1002, 17]
a,a,a,f,whisky,luxardo-amaretto-di-saschira,fresh-lemon-juice,simple-syrup,sparkling-apple-juice,seltzer
[1, 1, 1, 6, 16, 55, 28, 17, 1001, 1002]
[1, 1, 1, 6, 16]
[1, 1, 1, 6, 16, 55]
[1, 1, 1, 6, 16, 55, 28]
[1, 1, 1, 6, 16, 55, 28, 17]
[1, 1, 1, 6, 16, 55, 28, 17, 1001]
[1, 1, 1, 6, 16, 55, 28, 17, 1001, 1002]
a,a,a,f,whisky,luxardo-amaretto-di-saschira,fresh-lemon-juice,simple-syrup,seltzer,sparkling-apple-juice
[1, 1, 1, 6, 16, 55, 28, 17, 1002, 1001]
[1, 1, 1, 6, 16]
[1, 1, 1, 6, 16, 55]
[1, 1, 1, 6, 1

[2, 3, 1, 2, 8, 18, 1067, 1066, 187]
[2, 3, 1, 2, 8]
[2, 3, 1, 2, 8, 18]
[2, 3, 1, 2, 8, 18, 1067]
[2, 3, 1, 2, 8, 18, 1067, 1066]
[2, 3, 1, 2, 8, 18, 1067, 1066, 187]
b,d,a,b,gin,pineapple-juice,lemon-sour,lemon-lime-soda,monin-strawberry-puree
[2, 3, 1, 2, 8, 18, 1067, 187, 1066]
[2, 3, 1, 2, 8]
[2, 3, 1, 2, 8, 18]
[2, 3, 1, 2, 8, 18, 1067]
[2, 3, 1, 2, 8, 18, 1067, 187]
[2, 3, 1, 2, 8, 18, 1067, 187, 1066]
b,d,b,b,gin,lime-sour,orange-juice,dry-curacao
[2, 3, 2, 2, 8, 1068, 10, 254]
[2, 3, 2, 2, 8]
[2, 3, 2, 2, 8, 1068]
[2, 3, 2, 2, 8, 1068, 10]
[2, 3, 2, 2, 8, 1068, 10, 254]
b,d,b,b,gin,lime-sour,dry-curacao,orange-juice
[2, 3, 2, 2, 8, 1068, 254, 10]
[2, 3, 2, 2, 8]
[2, 3, 2, 2, 8, 1068]
[2, 3, 2, 2, 8, 1068, 254]
[2, 3, 2, 2, 8, 1068, 254, 10]
b,d,b,b,gin,orange-juice,lime-sour,dry-curacao
[2, 3, 2, 2, 8, 10, 1068, 254]
[2, 3, 2, 2, 8]
[2, 3, 2, 2, 8, 10]
[2, 3, 2, 2, 8, 10, 1068]
[2, 3, 2, 2, 8, 10, 1068, 254]
b,d,b,b,gin,orange-juice,dry-curacao,lime-sour
[2, 3, 2, 2, 8, 10, 25

[1, 1, 1, 3, 26, 258, 122, 24, 87]
a,a,a,d,cognac,rye-whiskey,tempus-fugit-alessio-vermouth-bianco,peychaud's-bitters,angostura-bitters
[1, 1, 1, 3, 26, 258, 122, 87, 24]
[1, 1, 1, 3, 26]
[1, 1, 1, 3, 26, 258]
[1, 1, 1, 3, 26, 258, 122]
[1, 1, 1, 3, 26, 258, 122, 87]
[1, 1, 1, 3, 26, 258, 122, 87, 24]
a,a,a,d,cognac,rye-whiskey,angostura-bitters,tempus-fugit-alessio-vermouth-bianco,peychaud's-bitters
[1, 1, 1, 3, 26, 258, 24, 122, 87]
[1, 1, 1, 3, 26]
[1, 1, 1, 3, 26, 258]
[1, 1, 1, 3, 26, 258, 24]
[1, 1, 1, 3, 26, 258, 24, 122]
[1, 1, 1, 3, 26, 258, 24, 122, 87]
a,a,a,d,cognac,rye-whiskey,angostura-bitters,peychaud's-bitters,tempus-fugit-alessio-vermouth-bianco
[1, 1, 1, 3, 26, 258, 24, 87, 122]
[1, 1, 1, 3, 26]
[1, 1, 1, 3, 26, 258]
[1, 1, 1, 3, 26, 258, 24]
[1, 1, 1, 3, 26, 258, 24, 87]
[1, 1, 1, 3, 26, 258, 24, 87, 122]
a,d,c,b,cognac,calvados,tempus-fugit-alessio-vermouth-bianco
[1, 3, 4, 2, 26, 428, 122]
[1, 3, 4, 2, 26]
[1, 3, 4, 2, 26, 428]
[1, 3, 4, 2, 26, 428, 122]
a,d,c,b,co

In [15]:
input_sequences[0:5]

[[1, 1, 3, 5, 7],
 [1, 1, 3, 5, 7, 1233],
 [1, 4, 3, 1, 148],
 [1, 4, 3, 1, 148, 149],
 [1, 1, 3, 4, 1234]]

### Add Pre-Padding so all sequences match

In [16]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences,   
                        maxlen=max_sequence_len, padding='pre'))

In [17]:
input_sequences[0:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    1,    1,    3,
           5,    7],
       [   0,    0,    0,    0,    0,    0,    0,    1,    1,    3,    5,
           7, 1233],
       [   0,    0,    0,    0,    0,    0,    0,    0,    1,    4,    3,
           1,  148],
       [   0,    0,    0,    0,    0,    0,    0,    1,    4,    3,    1,
         148,  149],
       [   0,    0,    0,    0,    0,    0,    0,    0,    1,    1,    3,
           4, 1234]])

In [21]:
max_sequence_len

13

### Split Input Sequences into Predictors and Labels

In [22]:
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_words)

## Create Model

In [23]:
def create_model(predictors, label, max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=input_len))
    model.add(Bidirectional(LSTM(150)))
    model.add(Dropout(0.1))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.fit(predictors, label, epochs=50, verbose=1)
    return model

In [ ]:
input_len = max_sequence_len - 1
model = Sequential()
model.add(Embedding(total_words, 10, input_length=input_len))
model.add(Bidirectional(LSTM(150)))
model.add(Dropout(0.1))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(predictors, label, epochs=50, verbose=1)

### Train Model

In [24]:
model = create_model(predictors, label, max_sequence_len, total_words)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
22047/22047 [==============================] - 28s 1ms/step - loss: 5.4560
Epoch 2/50
22047/22047 [==============================] - 30s 1ms/step - loss: 4.9621
Epoch 3/50
22047/22047 [==============================] - 31s 1ms/step - loss: 4.7874
Epoch 4/50
22047/22047 [==============================] - 29s 1ms/step - loss: 4.6095
Epoch 5/50
22047/22047 [==============================] - 36s 2ms/step - loss: 4.3361
Epoch 6/50
22047/22047 [==============================] - 33s 2ms/step - loss: 3.8892
Epoch 7/50
22047/22047 [==============================] - 50s 2ms/step - loss: 3.4708
Epoch 8/50
22047/22047 [==============================] - 41s 2ms/step - loss: 3.1218
Epoch 9/50
22047/22047 [==============================] - 26s 1ms/step - loss

### Saving and Importing Model

In [25]:
# Save model to disk
import pickle
# save the model to disk
# filename = 'vikram_model.sav'
# pickle.dump(model, open(filename, 'wb'))


model.save('cocktail_model.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model

In [2]:
# # load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))

# load json and create model
# returns a compiled model
# identical to the previous one
from keras.models import load_model

model = load_model('cocktail_model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [13]:
# Another saving method
model_json = model.to_json()

with open("cocktail_model_json","w") as json_file:
    json_file.write(model_json)
    
model.save_weights("cocktail_model_weights.h5")

# Post-Processing
## Cocktail Generating Functions

### Generate Potential List of Ingredients

In [3]:
# Generates potential ingredient list based on answers to the four questions (seed text)
def generate_text(seed_text, next_words, max_sequence_len, model):
    x = set()
    drink_list = []
    for j in range(next_words):
        # convert input into vectorized format
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        # prepare vector for input into model
        token_list = pad_sequences([token_list], maxlen= 
                             max_sequence_len-1, padding='pre')
        # make a prediction for what word could come after the input sequence
        predicted = model.predict_classes(token_list, verbose=0)
  
        output_word = ""
        # find the corresponding word to the vectorized output
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        # new seed_text is the predicted word + space
        seed_text += " " + output_word
        # if the output_word is unique from the growing list
        # add it to the set of predicted words
        # otherwise go to the next prediction
        if(output_word not in x):
            drink_list.append(output_word)
            x.add(output_word)
    return drink_list

### Create Cocktail from List of Potential Ingredients and Answers to 4 Questions

In [4]:
# Creates the final cocktail recipe using the potential ingredient list generated above and the answers to the four q's
def generate_drink_output(four_questions, ingredient_list):
    liquor = ingredient_list.pop(0) # extract liquor from list of ingredients
    ingredients = [3,4,5,7] # number of ingredients associated with each response to music question
    num_ing_indicator = ord(four_questions.split()[2]) - 96 # getting the number of ingredients based on music choice?
    num_ing = ingredients[num_ing_indicator-1] # number of ingredients
#     select_ingredients = random.choices(ingredient_list,k=num_ing) # randomly select number from ingredient list
    select_ingredients = list(np.random.choice(ingredient_list, size=num_ing, replace=False)) #
    final_drink = [liquor] + select_ingredients # final drink is the liquor plus the ingredients
    final_drink = [ing.replace('-',' ').strip() for ing in final_drink] # remove the dashes
    print("Here's your Drink:")
    print(', '.join(final_drink))

## Example Cocktail Creations

In [12]:
four_questions = "d c d d"
max_sequence_len = 13
ingredient_list = generate_text(four_questions, 30, max_sequence_len, model)
generate_drink_output(four_questions, ingredient_list)

Here's your Drink:
wine, luxardo aperitivo, frozen blueberries, sweet & sour mix, luxardo maraschino liqueur, lemon juice, bittermens xocolatl mole bitters, noilly prat sweet vermouth


In [63]:
import random
for i in range(10):
    four_questions = ' '.join(random.choices(['a','b','c','d'],k=4))
    ingredient_list = generate_text(four_questions, 30, max_sequence_len, model)
    generate_drink_output(four_questions, ingredient_list)
    print('\n')

Here's your Drink:
orange brandy liqueur, tripe sec, whipping cream, bitters, up


Here's your Drink:
whiskey, bacardi rum, sweet and sour mix, bacardi rum, bourbon, margarita mix, bourbon, triple sec


Here's your Drink:
pepe lopez tequila, luxardo aperitivo, agave nectar, lime juice, agave syrup, luxardo amaretto di saschira, sweet and sour, 1 agave to water)


Here's your Drink:
irish whiskey, milk, sour mix, bitters, green creme de menthe


Here's your Drink:
brandy, dark creme de cacao, dark creme de cacao, baileys


Here's your Drink:
tequila, pages elderflower syrup, angostura bitters, egg white, fever tree ginger beer, angostura bitters, angostura bitters, bin 27 port


Here's your Drink:
galliano, hot chocolate, white creme de cacao, amaretto, cream, baileys


Here's your Drink:
vodka, grenadine, soda, coffee liqueur, coffee liqueur, banana liqueur, crushed ice, crushed ice


Here's your Drink:
tequila, es celery bitters, grapefruit juice, grapefruit juice


Here's your Drink:

In [2]:
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


## Cocktail Widget

In [7]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output

q1 = widgets.RadioButtons(options=['Whiskey', 'Vodka', 'Tequila','Rum'],#     value='pineapple',
    description='',
    disabled=False)
q2 = widgets.RadioButtons(options=['Fall', 'Winter', 'Spring','Summer'],#     value='pineapple',
    description='',
    disabled=False)
q3 = widgets.RadioButtons(options=['Some Sinatra', 'Probably Rock', 'Big Chart-Toppers','Hot Club Hits'],#     value='pineapple',
    description='',
    disabled=False)
q4 = widgets.RadioButtons(options=['Frozen', 'Fruity', 'Hot', 'Sour','Sweet','Floral'],#     value='pineapple',
    description='',
    disabled=False)

accordion = widgets.Accordion(children=[q1, q2, q3, q4])
accordion.set_title(0, 'Pick a type of alcohol')
accordion.set_title(1, "What's your favorite season?")
accordion.set_title(2, "Your usual watering hole plays...")
accordion.set_title(3, 'Pick a flavor profile')
display(accordion)

ind = ['a','b','c','d','e','f','g','h'] # 0, 1, 2, 3
def update_answers(b):
    with out:
        clear_output()    
        a1 = ind[accordion.children[0].index]
        a2 = ind[accordion.children[1].index]
        a3 = ind[accordion.children[2].index]
        a4 = ind[accordion.children[3].index]
        txt_input = a1 + ' ' + a2 + ' ' + a3 + ' ' + a4
#         print(txt_input)
        list_input = generate_text(txt_input, 30, max_sequence_len, model)
        generate_drink_output(txt_input, list_input)

done = widgets.Button(
    description='Make me a drink!',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
done.on_click(update_answers)
display(done)

out = widgets.Output()
display(out)

Accordion(children=(RadioButtons(options=('Whiskey', 'Vodka', 'Tequila', 'Rum'), value='Whiskey'), RadioButton…

Button(button_style='info', description='Make me a drink!', icon='check', style=ButtonStyle(), tooltip='Click …

Output()